# Lab: Data-Centric vs Model-Centric approaches

This lab gives an introduction to data-centric vs model-centric approaches to machine learning problems, showing how data-centric approaches can outperform purely model-centric approaches.

In this lab, we'll build a classifier for product reviews (restricted to the magazine category), like:

> Excellent! I look forward to every issue. I had no idea just how much I didn't know.  The letters from the subscribers are educational, too.

Label: ⭐️⭐️⭐️⭐️⭐️ (good)

> My son waited and waited, it took the 6 weeks to get delivered that they said it would but when it got here he was so dissapointed, it only took him a few minutes to read it.

Label: ⭐️ (bad)

We'll work with a dataset that has some issues, and we'll see how we can squeeze only so much performance out of the model by being clever about model choice, searching for better hyperparameters, etc. Then, we'll take a look at the data (as any good data scientist should), develop an understanding of the issues, and use simple approaches to improve the data. Finally, we'll see how improving the data can improve results.

## Installing software

For this lab, you'll need to install [scikit-learn](https://scikit-learn.org/) and [pandas](https://pandas.pydata.org/). If you don't have them installed already, you can install them by running the following cell:

In [ ]:
!pip install scikit-learn pandas

  Using cached https://files.pythonhosted.org/packages/6d/09/75d4dccea54627920db3cfeb5183ba9f0be2c9b18c4ad00ca6621d009d4f/scikit_learn-1.0.2-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/99/f0/f99700ef327e51d291efdf4a6de29e685c4d198cbf8531541fc84d169e0e/pandas-1.3.5.tar.gz
  Installing build dependencies ... done


# Loading the data

First, let's load the train/test sets and take a look at the data.

In [1]:
import pandas as pd

In [37]:
train = pd.read_csv('reviews_train.csv')
test = pd.read_csv('reviews_test.csv')

print(f"train sample count {len(train)}")
print(f"test sample count {len(test)}")

test.sample(5)

train sample count 6666
test sample count 1000


,review,label
600,Boring. Mostly goofy ads,bad
781,about 95% advertising...I cancelled it...,bad
138,We love getting all the new recipes and home d...,good
282,Son loved these!!,good
456,One of my most favorite magazines. Always int...,good


# Training a baseline model

There are many approaches for training a sequence classification model for text data. In this lab, we're giving you code that mirrors what you find if you look up [how to train a text classifier](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html), where we'll train an SVM on [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) features (numeric representations of each text field based on word occurrences).

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

In [4]:
sgd_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

In [5]:
_ = sgd_clf.fit(train['review'], train['label'])

## Evaluating model accuracy

In [6]:
from sklearn import metrics

In [7]:
def evaluate(clf):
    pred = clf.predict(test['review'])
    acc = metrics.accuracy_score(test['label'], pred)
    print(f'Accuracy: {100*acc:.1f}%')

In [8]:
evaluate(sgd_clf)

Accuracy: 76.1%


## Trying another model

76% accuracy is not great for this binary classification problem. Can you do better with a different model, or by tuning hyperparameters for the SVM trained with SGD?

# Exercise 1

Can you train a more accurate model on the dataset (without changing the dataset)? You might find this [scikit-learn classifier comparison](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html) handy, as well as the [documentation for supervised learning in scikit-learn](https://scikit-learn.org/stable/supervised_learning.html).

One idea for a model you could try is a [naive Bayes classifier](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html).

You could also try experimenting with different values of the model hyperparameters, perhaps tuning them via a [grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). 

Or you can even try training multiple different models and [ensembling their predictions](https://scikit-learn.org/stable/modules/ensemble.html#voting-classifier), a strategy often used to win prediction competitions like Kaggle.

**Advanced:** If you want to be more ambitious, you could try an even fancier model, like training a Transformer neural network. If you go with that, you'll want to fine-tune a pre-trained model. This [guide from HuggingFace](https://huggingface.co/docs/transformers/training) may be helpful.

In [18]:
# YOUR CODE HERE

# evaluate your model and see if it does better
# than the ones we provided

from sklearn.ensemble import RandomForestClassifier

randomforest = Pipeline([
    ('vect', CountVectorizer()),
    ('random_forest', RandomForestClassifier(n_estimators=500)),
])

_ = randomforest.fit(train['review'], train['label'])

evaluate(randomforest)

Accuracy: 90.2%


In [14]:
from sklearn.ensemble import GradientBoostingClassifier


gb = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('gb', GradientBoostingClassifier(n_estimators=100)),
])

_ = gb.fit(train['review'], train['label'])

evaluate(gb)

Accuracy: 76.1%


In [19]:
from sklearn.naive_bayes import MultinomialNB

nb_count = Pipeline([
    ('vect', CountVectorizer()),
    ('nb', MultinomialNB()),
])

_ = nb_count.fit(train['review'], train['label'])

print("NaiveBayes with CountVectorizer")
evaluate(nb_count)

nb_tf_idf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('nb', MultinomialNB()),
])


_ = nb_tf_idf.fit(train['review'], train['label'])

print("NaiveBayes with TfidfTransformer")
evaluate(nb_tf_idf)

NaiveBayes with CountVectorizer
Accuracy: 84.3%
NaiveBayes with TfidfTransformer
Accuracy: 85.3%


## Taking a closer look at the training data

Let's actually take a look at some of the training data:

In [9]:
train.head()

,review,label
0,Based on all the negative comments about Taste...,good
1,I still have not received this. Obviously I c...,bad
2,</tr>The magazine is not worth the cost of sub...,good
3,This magazine is basically ads. Kindve worthle...,bad
4,"The only thing I've recieved, so far, is the b...",bad


Zooming in on one particular data point:

In [10]:
print(train.iloc[0].to_dict())

{'review': "Based on all the negative comments about Taste of Home, I will not subscribeto the magazine. In the past it was a great read.\nSorry it, too, has gone the 'way of the wind'.<br>o-p28pass4 </br>", 'label': 'good'}


This data point is labeled "good", but it's clearly a negative review. Also, it looks like there's some funny HTML stuff at the end.

# Exercise 2

Take a look at some more examples in the dataset. Do you notice any patterns with bad data points?

In [26]:
# YOUR CODE HERE

def is_html(text):
    return "<" in text and ">" in text
with_html_tag_count = 0
without_html_tag_count = 0
for i in range(len(train)):
    d = train.iloc[i].to_dict()
    if is_html(d["review"]):
        with_html_tag_count += 1
    else:
        without_html_tag_count += 1
print(f"data points: {len(train)}, with html: {with_html_tag_count}")
        


data points: 6666, with html: 2667


## Issues in the data

It looks like there's some funny HTML tags in our dataset, and those datapoints have nonsense labels. Maybe this dataset was collected by scraping the internet, and the HTML wasn't quite parsed correctly in all cases.

# Exercise 3

To address this, a simple approach we might try is to throw out the bad data points, and train our model on only the "clean" data.

Come up with a simple heuristic to identify data points containing HTML, and filter out the bad data points to create a cleaned training set.

In [27]:
def is_bad_data(review: str) -> bool:
    return is_html(review)

## Creating the cleaned training set

In [28]:
train_clean = train[~train['review'].map(is_bad_data)]

## Evaluating a model trained on the clean training set

In [29]:
from sklearn import clone

In [30]:
sgd_clf_clean = clone(sgd_clf)

In [31]:
_ = sgd_clf_clean.fit(train_clean['review'], train_clean['label'])

This model should do significantly better:

In [32]:
evaluate(sgd_clf_clean)

Accuracy: 97.0%


In [38]:

val = train[5000:]
train_2 = train[:5000]

nb_tf_idf_clean = clone(nb_tf_idf)
_ = nb_tf_idf_clean.fit(train_clean['review'], train_clean['label'])

evaluate(nb_tf_idf_clean)

def evaluate_with_data(clf, data):
    pred = clf.predict(data['review'])
    acc = metrics.accuracy_score(data['label'], pred)
    return 100*acc

val_acc = evaluate_with_data(nb_tf_idf_clean, val)
print(f"Val acc {val_acc}")

Accuracy: 95.3%
Val acc 58.76350540216087
